In [2]:
import xarray as xr
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# extracting post treatment data to be used in model application

In [3]:
sfe = xr.open_dataset("/scratch/users/ashdef/filtered_sfe_gridmet/sfe_filtered.nc")
precip = xr.open_dataset("/scratch/users/ashdef/filtered_sfe_gridmet/precip_filtered.nc")
rmax = xr.open_dataset("/scratch/users/ashdef/filtered_sfe_gridmet/rmax_filtered.nc")
rmin = xr.open_dataset("/scratch/users/ashdef/filtered_sfe_gridmet/rmin_filtered.nc")
srad = xr.open_dataset("/scratch/users/ashdef/filtered_sfe_gridmet/srad_filtered.nc")
tmax = xr.open_dataset("/scratch/users/ashdef/filtered_sfe_gridmet/tmax_filtered.nc")
tmin = xr.open_dataset("/scratch/users/ashdef/filtered_sfe_gridmet/tmin_filtered.nc")
vpd = xr.open_dataset("/scratch/users/ashdef/filtered_sfe_gridmet/vpd_filtered.nc")
ws = xr.open_dataset("/scratch/users/ashdef/filtered_sfe_gridmet/ws_filtered.nc")

In [4]:
sfe

<xarray.Dataset> Size: 472MB
Dimensions:  (time: 468, lat: 350, lon: 360)
Coordinates:
  * lat      (lat) float64 3kB 49.03 48.94 48.9 48.82 ... 31.65 31.52 31.4 30.65
  * lon      (lon) float64 3kB -124.4 -124.4 -124.3 ... -90.64 -83.72 -83.68
  * time     (time) datetime64[ns] 4kB 1985-01-01 1985-02-01 ... 2023-12-01
Data variables:
    ET       (time, lat, lon) float64 472MB ...

In [5]:
# renaming so [var] matches name of dataset
precip = precip.rename({'precipitation_amount': 'precip'})
rmax = rmax.rename({'relative_humidity' : 'rmax'})
rmin= rmin.rename({'relative_humidity' : 'rmin'})
srad = srad.rename({'surface_downwelling_shortwave_flux_in_air' : 'srad'})
tmin = tmin.rename({'air_temperature' : 'tmin'})
tmax = tmax.rename({'air_temperature' : 'tmax'})
vpd = vpd.rename({'mean_vapor_pressure_deficit' : 'vpd'})
ws = ws.rename({'wind_speed' : 'ws'})

In [6]:
pixel_treatment = pd.read_csv('pixel_treatment_clean.csv')
pixel_treatment = pixel_treatment.sort_values('treatment_date')
pixel_treatment.reset_index()

,index,Unnamed: 0,pixel_id,lat,lon,pixel_geometry,index_right,treatment_date,twig_categ,treatment_geometry
0,2190,116676,"(84, 252)",45.900000,-114.266667,POLYGON ((-12717787.98820922 5761006.692671617...,5439,1986-01-01,Mechanical,MULTIPOLYGON Z (((-12717456.6514 5760763.97770...
1,6039,234246,"(169, 12)",42.358333,-124.266667,POLYGON ((-13830982.896141954 5211670.14706264...,46058,1986-01-01,Mechanical,POLYGON Z ((-13834012.5258 5218561.729099996 -...
2,3344,152619,"(110, 159)",44.816667,-118.141667,POLYGON ((-13149151.015033154 5589436.68652023...,44196,1986-01-07,Mechanical,POLYGON Z ((-13154055.344700001 5588978.509900...
3,10973,482615,"(348, 287)",34.900000,-112.808333,POLYGON ((-12555447.064135693 4147472.47688450...,23130,1986-02-01,Mechanical,POLYGON Z ((-12558898.384 4150060.4245000035 -...
4,4535,184401,"(133, 63)",43.858333,-122.141667,POLYGON ((-13594428.978206249 5440329.27497087...,35229,1986-02-01,Mechanical Mechanical,MULTIPOLYGON Z (((-13598303.988699999 5443431....
...,...,...,...,...,...,...,...,...,...,...
12555,10404,438246,"(316, 270)",36.233333,-113.516667,POLYGON ((-12634298.370114263 4329900.57739480...,54740,2023-12-29,Mechanical,MULTIPOLYGON Z (((-12638319.9999 4336078.2817 ...
12556,10386,436860,"(315, 270)",36.275000,-113.516667,POLYGON ((-12634298.370114263 4335650.90843343...,54740,2023-12-29,Mechanical,MULTIPOLYGON Z (((-12638319.9999 4336078.2817 ...
12557,10385,436859,"(315, 269)",36.275000,-113.558333,POLYGON ((-12638936.68223065 4335650.908433433...,54740,2023-12-29,Mechanical,MULTIPOLYGON Z (((-12638319.9999 4336078.2817 ...
12558,2090,112491,"(81, 225)",46.025000,-115.391667,POLYGON ((-12843022.415351653 5781016.95665349...,67564,2023-12-30,Mechanical Mechanical,MULTIPOLYGON Z (((-12846109.4089 5781947.29540...


### drop pixels with 2023 treatment date - no valid post-treatment ET data if using calendar year buffer

https://www.geeksforgeeks.org/pandas/drop-rows-from-the-dataframe-based-on-certain-condition-applied-on-a-column/

In [7]:
trmt_2023 = 0

pixel_treatment['treatment_date'] = pd.to_datetime(pixel_treatment['treatment_date'])

for i in range(len(pixel_treatment)):
    pixel = pixel_treatment.index[i]
    if pixel_treatment['treatment_date'].loc[pixel].year == 2023:
        trmt_2023+=1

print(trmt_2023)

1016


In [9]:
treated_2023_indices = pixel_treatment[pixel_treatment['treatment_date'].dt.year == 2023].index

pixel_treatment.drop(treated_2023_indices, inplace = True)
        

In [10]:
pixel_treatment

,Unnamed: 0,pixel_id,lat,lon,pixel_geometry,index_right,treatment_date,twig_categ,treatment_geometry
2190,116676,"(84, 252)",45.900000,-114.266667,POLYGON ((-12717787.98820922 5761006.692671617...,5439,1986-01-01,Mechanical,MULTIPOLYGON Z (((-12717456.6514 5760763.97770...
6039,234246,"(169, 12)",42.358333,-124.266667,POLYGON ((-13830982.896141954 5211670.14706264...,46058,1986-01-01,Mechanical,POLYGON Z ((-13834012.5258 5218561.729099996 -...
3344,152619,"(110, 159)",44.816667,-118.141667,POLYGON ((-13149151.015033154 5589436.68652023...,44196,1986-01-07,Mechanical,POLYGON Z ((-13154055.344700001 5588978.509900...
10973,482615,"(348, 287)",34.900000,-112.808333,POLYGON ((-12555447.064135693 4147472.47688450...,23130,1986-02-01,Mechanical,POLYGON Z ((-12558898.384 4150060.4245000035 -...
4535,184401,"(133, 63)",43.858333,-122.141667,POLYGON ((-13594428.978206249 5440329.27497087...,35229,1986-02-01,Mechanical Mechanical,MULTIPOLYGON Z (((-13598303.988699999 5443431....
...,...,...,...,...,...,...,...,...,...
7423,292790,"(211, 344)",40.608333,-110.433333,POLYGON ((-12291063.273501668 4951687.08824967...,27122,2022-12-22,Unknown,POLYGON Z ((-12295149.8972 4954001.159599997 -...
4173,173637,"(125, 387)",44.191667,-108.641667,POLYGON ((-12091615.852497054 5491917.20746013...,54110,2022-12-27,Chemical,MULTIPOLYGON Z (((-12093931.5276 5495225.02449...
4174,173638,"(125, 388)",44.191667,-108.600000,POLYGON ((-12086977.540380666 5491917.20746013...,54110,2022-12-27,Chemical,MULTIPOLYGON Z (((-12093931.5276 5495225.02449...
727,50058,"(36, 162)",47.900000,-118.016667,POLYGON ((-13135236.078683995 6086775.96415573...,52676,2022-12-28,Mechanical,MULTIPOLYGON Z (((-13135594.3514 6088755.27740...


In [18]:
pixel_treatment.to_csv('/scratch/users/ashdef/post_treatment_data/pixel_treatment.csv')

### post treatment arrays

In [11]:
# use treatment date to build post-treatment arrays - used help from chatGPT
pixel_treatment['treatment_date'] = pd.to_datetime(pixel_treatment['treatment_date'])

climate_datasets = {
    'precip': precip,
    'rmax': rmax,
    'rmin': rmin,
    'srad': srad,
    'tmin': tmin,
    'tmax': tmax,
    'vpd': vpd,
    'ws': ws
}

# lists of arrays for each pixel
climate_list = []
sfe_list = []

for _, row in pixel_treatment.iterrows(): #looping over pixels
    lat = row['lat']
    lon = row['lon']
    t_date = row['treatment_date']

    sfe_pixel = sfe.sel(lat=lat, lon=lon, method = 'nearest') # need to use nearest because roundings are different

    # subset post-treatment
    post_mask = sfe_pixel['time'].dt.year > t_date.year

    sfe_post = sfe_pixel['ET'].sel(time=post_mask).values

    clim_arrays = [] # stack all climate variables for this pixel
    for var, ds in climate_datasets.items(): # looping over each gridmet var
        clim_pixel = ds.sel(lat=lat, lon=lon, method = 'nearest')
        clim_post = clim_pixel[var].sel(time = post_mask).values # need values so it's not an object from xarray
        clim_arrays.append(clim_post)

    clim_post = np.stack(clim_arrays, axis = -1) # shape (timesteps, number of features)

    climate_list.append(clim_post)
    sfe_list.append(sfe_post)


# wrap the lists as object arrays - allows arrays of different lengths to be saved
sfe_obj = np.array(sfe_list, dtype=object)
climate_obj = np.array(climate_list, dtype=object)

np.savez_compressed("/scratch/users/ashdef/post_treatment_data/sfe_post_filtered.npz", sfe_list=sfe_obj)
np.savez_compressed("/scratch/users/ashdef/post_treatment_data/climate_post_filtered.npz", climate_list=climate_obj)


In [12]:
sfe_post = np.load("/scratch/users/ashdef/post_treatment_data/sfe_post_filtered.npz", allow_pickle = True)["sfe_list"] 
climate_post = np.load("/scratch/users/ashdef/post_treatment_data/climate_post_filtered.npz", allow_pickle = True)["climate_list"]

In [13]:
sfe_post.shape

(11544,)

### 12/2023 has nan at all timesteps for every pixel, so delete the last index of each array


In [14]:
sfe_post[0][-1]

nan

In [15]:
for pixel in range(len(sfe_post)):
    sfe_post[pixel] = sfe_post[pixel][:-1]
    climate_post[pixel] = climate_post[pixel][:-1]
sfe_post[0].shape

(443,)

In [16]:
num_pixels_with_nan = 0
for i in range(len(sfe_post)):
    if np.isnan(sfe_post[i]).sum() >0: #and np.isnan(sfe_post[i]).sum() != 1:
        print(i, np.isnan(sfe_post[i]).sum())
        num_pixels_with_nan +=1
print(num_pixels_with_nan, len(sfe_post))

234 431
682 2
770 1
899 1
904 1
1101 1
1110 1
2256 251
2257 251
2476 239
2510 239
2530 239
2728 227
2752 215
3148 203
3149 203
3151 203
3152 203
3500 191
3676 179
3677 179
3679 179
3687 179
3690 179
3697 179
3698 179
3699 179
3726 179
3788 179
3798 179
3861 179
4164 167
4221 167
4223 167
4321 167
5162 143
5168 143
5528 143
5644 131
5923 131
6068 119
6069 119
6172 119
6238 119
6239 119
6552 107
6554 107
6579 107
6633 107
6635 107
6636 107
6762 107
7230 95
7382 83
7444 83
7703 83
7830 83
7872 83
8611 59
8805 59
8808 59
9072 47
9263 47
9266 47
9651 35
9652 35
9666 35
9668 35
10264 23
10267 23
10269 23
10270 23
10271 23
10304 23
10338 23
10650 23
10755 11
11520 11
11532 11
79 11544


In [17]:
np.savez_compressed("/scratch/users/ashdef/post_treatment_data/sfe_post_filtered.npz", sfe_list=sfe_post)
np.savez_compressed("/scratch/users/ashdef/post_treatment_data/climate_post_filtered.npz", climate_list=climate_post)